In [0]:
pip install ndlib

In [0]:
pip install cdlib

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from cdlib import algorithms
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import pandas as pd

In [0]:
def ICM(graph, per_inf, threshold, num_iter):
  model = ep.IndependentCascadesModel(graph)

  config = mc.Configuration()
  config.add_model_parameter('percentage_infected', per_inf)
  for e in graph.edges():
      config.add_edge_configuration("threshold", e, threshold) # edge attribute setting

  model.set_initial_status(config)

  iterations = model.iteration_bunch(num_iter, node_status=True)

  trends = model.build_trends(iterations)
  return iterations

In [0]:
fils = ['/content/drive/My Drive/Datasets/ca-HepPh.mtx'
,'/content/drive/My Drive/Datasets/ca-GrQc.mtx'
,'/content/drive/My Drive/Datasets/econ-psmigr1.mtx'
,'/content/drive/My Drive/Datasets/ca-CSphd.mtx'
,'/content/drive/My Drive/Datasets/ca-sandi_auths.mtx'
,'/content/drive/My Drive/Datasets/ca-netscience.mtx'
,'/content/drive/My Drive/Datasets/bio-celegans.mtx'
,'/content/drive/My Drive/Datasets/bio-diseasome.mtx'
,'/content/drive/My Drive/Datasets/rt-twitter-copen.mtx'
,'/content/drive/My Drive/Datasets/econ-mbeause.mtx'
,'/content/drive/My Drive/Datasets/econ-wm2.mtx'
,'/content/drive/My Drive/Datasets/econ-mbeacxc.mtx']



In [0]:
def iter_analysis(z,length):
  count_inf = count_rem = avg_inf = avg_rem = 0
  leny = length
  for i in range(0,len(z)):
    x = z[i].get('status')
    for key in x:
      y =x[key]
      if y == 1:
        count_inf += 1
      elif y == 2:
        count_rem += 1
  avg_inf = count_inf / leny
  avg_rem = count_rem / leny
  return[count_inf, avg_inf, count_rem, avg_rem]

In [0]:
def speed_analysis(z,lenG,perc,v):
  time = 0
  count_inf = 0
  thres = perc*lenG
  for i in range(0,len(z)):
    x = z[i].get('status')
    for key in x:
      y =x[key]
      if y == 1:
        count_inf += 1
        if (count_inf >= thres and time == 0):
          time = i
  if time == 0:
    time = v + 1
  return time

In [0]:
def icm_data(a, b, num):
  inf_num = avg_inf = fin_rem = avg_rem = y1 = y2 = y3 = 0
  df = pd.DataFrame(columns = ['Graph', 'Model', 'Size', 'Infected Number', 'Avg_inf', 'Fin_rem', 'Avg_rem', 'Density', '40% Diffusion', '60% Diffusion', '80% Diffusion' ])
  lenG = G.order()
  n = 5
  for i in range(1,n):
    z = ICM(G, a, b, num)
    x = iter_analysis(z, lenG)
    z = ICM(G, a, b, 100)
    y1 += speed_analysis(z,lenG,0.4,100)
    y2 += speed_analysis(z,lenG,0.6,100)
    y3 += speed_analysis(z,lenG,0.8,100)
    inf_num += x[0]
    avg_inf += x[1]
    fin_rem += x[2]
    avg_rem += x[3] 
  df = df.append({'Graph' : 'Full Graph', 'Model' : 'ICM', 'Size' : lenG, 'Infected Number' : inf_num/n ,'Avg_inf' :avg_inf/n, 'Fin_rem' : fin_rem/n, 'Avg_rem' : avg_rem/n, 'Density' : fg_density, '40% Diffusion' : y1/n, '60% Diffusion' : y2/n, '80% Diffusion': y3/n} , ignore_index=True)
  for i in range (0, len(coms.communities)):
    sub = coms.communities[i]
    x = G.subgraph(sub)
    lenG = len(x)
    inf_num = avg_inf = fin_rem = avg_rem = 0
    for k in range(1,n):
      z = ICM(x, a, b, num)
      x = iter_analysis(z, lenG) 
      inf_num += x[0]
      avg_inf += x[1]
      fin_rem += x[2]
      avg_rem += x[3] 
    df = df.append({'Graph' : 'Sub Graph', 'Model' : 'ICM', 'Size' : lenG, 'Infected Number' : inf_num/n ,'Avg_inf' :avg_inf/n, 'Fin_rem' : fin_rem/n, 'Avg_rem' : avg_rem/n, 'Density' : sg_dens[i]} , ignore_index=True)
  return df

In [0]:
def aggegate_data_ICM():
  av = pd.DataFrame(columns = ['Model', 'Seed nodes', 'Inf_Threshold', 'Graph_size', 'FG_infection', 'SG_infection', 'Density', '40% Diffusion', '60% Diffusion', '80% Diffusion'])
  a = 0.1
  b = 0.2
  num = 10
  perc = np.arange(0.1, 0.4, 0.1)
  for a in perc:
    for b in perc:
      df = icm_data(a, b, num)
      den = df.loc[0, 'Density']
      model = df.loc[0, 'Model']
      y1 = df.loc[0, '40% Diffusion']
      y2 = df.loc[0, '60% Diffusion']
      y3 = df.loc[0, '80% Diffusion']
      lenG = G.order()
      FG = df.loc[0,'Infected Number'] / lenG
      count = df.loc[1:,'Infected Number'].sum() / lenG
      av = av.append({'Model' : model,'Seed nodes' : a, 'Inf_Threshold' : b, 'Graph_size' : lenG, 'FG_infection' : FG, 'SG_infection' : count, 'Density' : den, '40% Diffusion': y1, '60% Diffusion' : y2, '80% Diffusion' : y3} , ignore_index=True)
  return av

In [0]:
#### Main Class ####
i = 0
for f in fils:
  i+= 1
  print(i)
  G = nx.read_adjlist(f)
  fg_density = nx.density(G)
  sg_conn = []
  sg_dens = []
  coms = algorithms.louvain(G, weight='weight', resolution=3., randomize=False)
  for i in range (0, len(coms.communities)):
    sub = coms.communities[i]
    x = G.subgraph(sub)
    sg_dens.append(nx.density(x))
  data = aggegate_data_ICM()
  data.to_csv('ICM.csv')
  files.download('ICM.csv')